In [253]:
N = 100; # Number of atomic sites
M = int((N)/2); # Number of atomic sites on each side of the origin
a = 1.0 # Real space distance between each site
lattx = np.arange(-a*M,a*M,a); # '+1' because np.arange doesn't include the endpoint
latty = np.zeros(lattx.size);
lattice = figure(plot_height=100, plot_width=700, title="1D atomic chain",x_range=[-1.1*a*M,1.1*a*M]);
source1 = ColumnDataSource(data = {'xVal': lattx, 'yVal': latty});
lattice.circle('xVal','yVal',source = source1, size = 4, line_color = 'black', fill_color = 'black');
lattice.xaxis.axis_label = 'Real space distance (arb. unit)';
show(lattice)

In [256]:
k0 = 2*np.pi/(N*a);
k = np.linspace(-M*k0,(M-1)*k0,N);
energy = -2*np.cos(k);

In [257]:
# Calcualte resulting dispersion relation using periodic boundary condition (PC)
band = figure(plot_height = 400, plot_width = 700, title='Energy band in reciprocal space', x_range = [-1.1*k0*M/np.pi,1.1*k0*M/np.pi], y_range= [1.2*energy.min(),1.2*energy.max()]);
source2 = ColumnDataSource(data = {'xVal': k/np.pi, 'yVal' : energy});
band.circle('xVal','yVal',source = source2,size = 4);
band.line('xVal','yVal',source = source2);
band.xaxis.axis_label='k-vector ( \u03c0 )';
band.yaxis.axis_label='Energy (arb. Unit)';
show(band)

In [261]:
# define the initial wavefunction (with an artificial Gaussian dist. in k-space)
sigm = 4*k0;
kk0 = 25*k0;
kk = np.linspace(-M*k0,(M-1)*k0, 200);
phi_k0 = 1/(np.sqrt(2*np.pi)*sigm)*np.exp(-1/2*( (kk-kk0)/sigm )**2);

In [262]:
# Initialize an electron at the center of the 1D chain with a gaussian distribution to indicates its probability density
dist0 = figure(plot_height = 400, plot_width = 700, title='Initial wavefunction in real space', x_range = [-1.1*k0*M/np.pi,1.1*k0*M/np.pi], y_range= [1.2*energy.min(),1.2*energy.max()]);
source3 = ColumnDataSource(data = {'xVal': kk/np.pi, 'yVal': phi_k0/phi_k0.max()});
dist0.line('xVal','yVal', source = source3, line_color = 'red');
dist0.circle('xVal','yVal',source = source2, size = 4, line_color = 'black', fill_color = 'black');
dist0.xaxis.axis_label='k-space distance (arb. unit)';
dist0.yaxis.axis_label='Probability density';
show(dist0)

In [263]:
phi_k0 = 1/(np.sqrt(2*np.pi)*sigm)*np.exp(-1/2*( (k-kk0)/sigm )**2);
phi_r0 = np.empty(N, dtype = complex);
for nn in range(0,N):
    psi0 = 0;
    for kn in range(k.size):
        psi0 = psi0 + phi_k0[kn]*np.exp(-1j*k[kn]*nn*a);
    phi_r0[nn] = psi0;
    rho0[nn] = np.abs(psi0);

IndexError: index 50 is out of bounds for axis 0 with size 50

In [264]:
# Construct hamiltonian using occupation number basis and set initial wavefunction
asd = np.zeros((N-1,N-1),complex);
np.fill_diagonal(asd,-1);
b = np.append(asd,np.zeros((N-1,1),complex), axis = 1);
c = np.append(np.zeros((1,N),complex),b,axis = 0);
hamil = c + np.transpose(c);
np.fill_diagonal(hamil,0);
psi0 = np.zeros((N,1),complex);
rho0 = np.zeros((N,1), float);

phi_k0 = 1/(np.sqrt(2*np.pi)*sigm)*np.exp(-1/2*( (k-kk0)/sigm )**2);
for nn in range(0,psi0.size):
    psi_temp = 0;
    for kn in range(k.size):
        psi_temp = psi_temp + phi_k0[kn]*np.exp(-1j*(k[kn]*(nn-M)*a));
    psi0[nn][0] = psi_temp;
    rho0[nn][0] = np.abs(psi0[nn][0]);

hamil[N-1][0] = -1; #Apply periodic boundry condition
hamil[0][N-1] = -1;
print(hamil)

[[ 0.+0.j -1.+0.j  0.+0.j ...  0.+0.j  0.+0.j -1.+0.j]
 [-1.+0.j  0.+0.j -1.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ... -1.+0.j  0.+0.j -1.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j ...  0.+0.j -1.+0.j  0.+0.j]]


In [265]:
# Define time evolution operator
dt = 0.1; # Define time step
endt = 6000;
diaganol = np.ones(N);
evol = np.diag(diaganol)+1.j*hamil*dt; # Use first order Taylor expansion to approximate the evolution operator
ts = np.arange(0,endt*dt,dt);
psis = np.empty(ts.size,dtype = object); # Initiate a container for the wavefunctions at different time slice
phis = np.empty(ts.size,dtype = object); # Initiate a container for the phase at different time slice
rhos = np.empty(ts.size,dtype = object); # Initiate a container for the probability density at different time slice
psis[0]=psi0;
for ii in range(1,ts.size):
    psis[ii] = np.dot(evol,psis[ii-1]);
lattx2 = np.arange(-M*a,a*(M-1),a/10.0);
for ii in range(0,ts.size-1):
    phis[ii] = np.angle((psis[ii]));
    rhos[ii] = np.abs(psis[ii]);

In [270]:
# Plot the phase of the wavefunction at time = t*dt
t = 0;
phi_t = np.squeeze(phis[t]);
pt = interp1d(lattx,np.squeeze(phis[t]), kind = 'cubic');
rho_t = interp1d(lattx,np.squeeze(rhos[t]), kind = 'cubic');
wavefunc_t = figure(plot_height=300, plot_width=700, title="Electron wavefunction",x_range=[-1.1*a*M,1.1*a*M]);
source4 = ColumnDataSource(data = {'xVal': lattx, 'yVal' : latty, 'cVal': phi_t});
lattx_smooth = np.linspace(-a*M,a*(M-1),num = 200, endpoint = True);
source5 = ColumnDataSource(data = {'xVal_s': lattx_smooth, 'yVal_s': rho_t(lattx_smooth)/rho_t(lattx_smooth).max()});

#phis_smooth = np.empty(ts.size,dtype = object); # Initiate a container for the phase at different time slice
#for ii in range(0,ts.size-1): # Option 1: use smoothed curve to show phase variation
#    phis_smooth[ii] = interp1d(lattx,np.squeeze(phis[ii]), kind = 'cubic');
#wavefunc_t.line('xVal','yVal', source = source4); 

mapper = linear_cmap(field_name = 'cVal', palette = RdYlBu11, low = min(phi_t), high = max(phi_t));
wavefunc_t.circle('xVal','yVal',source = source4, size = 4, line_color = mapper, fill_color = mapper);
wavefunc_t.line('xVal_s','yVal_s',source = source5, line_color = 'red');
color_bar = ColorBar(color_mapper = mapper['transform'], width = 300, location = (150,0), orientation = "horizontal");
wavefunc_t.add_layout(color_bar, 'below');
show(wavefunc_t)

In [267]:
def update(time):
    time = int(time/dt);
    pt = interp1d(lattx,np.squeeze(phis[t]), kind = 'cubic');
    rho_t = interp1d(lattx,np.squeeze(rhos[time]), kind = 'cubic');
    source4.data['cVal'] = np.squeeze(phis[time]);
    source5.data['yVal_s'] = rho_t(lattx_smooth)/rho_t(lattx_smooth).max();
    
    push_notebook(wavefunc_t)

In [268]:
show(wavefunc_t, notebook_handle = True);

In [269]:
interact(update, time = ipw.FloatSlider(min = 0, max = (endt-2)*dt, step = dt, value = 0));

interactive(children=(FloatSlider(value=0.0, description='time', max=599.8000000000001), Output()), _dom_class…

In [ ]:
#This animation is not functioning due to the current incompatibility between bokeh and tornado, should be resolved in furture updates.

# def auto_update():
#     global time;
#     source_tt.data['cVal'] = np.squeeze(phis[t])
#     source_tt.data['yVal'] = interp1d(lattice,np.squeeze(rhos[t]), kind = 'cubic')
#     source_tt.trigger('data', source_tt.data, source_tt.data)
#     if time < endt.size - 1:
#         time = time + 1
#     else:
#         time = 0
#         source_tt.data['cVal'] = np.squeeze(phis[0])
#         source_tt.data['yVal'] = interp1d(lattice,np.squeeze(rhos[0]), kind = 'cubic')

In [ ]:
#This animation is not functioning due to the current incompatibility between bokeh and tornado, should be resolved in furture updates.

# session = push_session(curdoc())
# time = 0; # set initial frame
# source_tt = ColumnDataSource(data = {'xVal': lattx, 'yVal' : rhos[0], 'cVal': np.zeros(lattx.size)})
# wavefunc_tt = figure(plot_height=300, plot_width=700, title="Evolution of electron wavefunction",x_range=[-1.1*a*M,1.1*a*M]);

# mapper = linear_cmap(field_name = 'cVal', palette = Spectral6, low = 0, high = 1);
# wavefunc_tt.line(x = 'xVal', y = 'yVal', source = source_tt, line_color = 'red');
# wavefunc_tt.circle('xPos','yPos',source = source4, size = 10, line_color = mapper, fill_color = mapper);

# curdoc().add_periodic_callback(auto_update, 67)

# session.show()
# session.loop_until_closed()